In [30]:
import os
import openai
from dotenv import find_dotenv, load_dotenv

# Load environment variables
_ = load_dotenv(find_dotenv())

# Set OpenAI API key
openai.api_type = os.getenv("api_type")
openai.api_base = os.getenv("api_base")
openai.api_version = os.getenv("api_version")
openai.api_key = os.getenv("OPENAI_API_KEY")

# RCI Chain with ChatModel

## Multi Chain

In [31]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import OpenAI

from langchain.schema.output_parser import StrOutputParser

In [32]:
model = AzureChatOpenAI(
        deployment_name="chatgpt-gpt35-turbo",
        model_name="gpt-35-turbo",
        temperature=0.7,
        max_tokens=1000
    )

In [33]:
prompt = ChatPromptTemplate.from_template(
    "tell me an intersting fact about {subject}"
    )

reverse_prompt = ChatPromptTemplate.from_template(
    "based on this interesting fact which is chunked down from a meta subject:\n\n {interesting_fact}\n\n Recover what the meta subject is\n Subject:"
    )

In [34]:
chain = prompt | model | StrOutputParser()

In [35]:
chain.invoke({"subject": "Laptop"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "subject": "Laptop"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "subject": "Laptop"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "tell me an intersting fact about Laptop",
          "additional_kwargs": {}
        }
      }
    ]
  }
}
[llm/start] [1:chain:RunnableSequence > 3:llm:AzureChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: tell me an intersting fact about Laptop"
  ]
}
[llm/end] [1:chain:RunnableSe

'The first laptop computer, the Grid Compass, was developed in 1982 and weighed 11 pounds, which was considered lightweight at the time.'

In [36]:
import langchain

langchain.debug = True

In [37]:
chain1 = prompt | model | StrOutputParser()

chain2 = {"interesting_fact": chain1} | reverse_prompt | model | StrOutputParser()


In [38]:
chain2.invoke({"subject": "Laptop"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "subject": "Laptop"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap] Entering Chain run with input:
{
  "input": {
    "subject": "Laptop"
  }
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "subject": "Laptop"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "subject": "Laptop"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages

[llm/end] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 5:llm:AzureChatOpenAI] [2.70s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The first laptop ever made was the Epson HX-20 in 1981, which was also the first computer to be called a \"laptop.\" It had a 4-line LCD screen and could run for up to 20 hours on four AA batteries.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The first laptop ever made was the Epson HX-20 in 1981, which was also the first computer to be called a \"laptop.\" It had a 4-line LCD screen and could run for up to 20 hours on four AA batteries.",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "

'History of Laptops'

# Testing RCI

In [39]:
langchain.debug = False

In [40]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


In [41]:
template="You are a helpful assistant that imparts wisdom and guides people with accurate answers."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [42]:
chain1 = chat_prompt | model | StrOutputParser()

In [43]:
initial_question = "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"

In [44]:
initial_answer = chain1.invoke({"question": initial_question})
initial_answer

"Roger has a total of 11 tennis balls now. \n\nHere's how you can break it down: \n\n- Roger starts with 5 tennis balls. \n- He buys 2 cans of tennis balls, so he adds 2 x 3 = 6 tennis balls to his collection. \n- In total, Roger now has 5 + 6 = 11 tennis balls."

In [45]:
fake_initial_ai_answer = """Roger initially has 5 tennis balls. Each can of tennis balls contains 3 tennis balls, and he bought 2 cans, so he has 2 x 3 = 6 additional tennis balls.
Therefore, the total number of tennis balls Roger has now is 5 + 4 = 9."""

## Part 2 - Critique  

In [46]:
template="You are a helpful assistant that looks at answers and finds what is wrong with them based on the original question given."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="### Question:\n\n{question}\n\n ###Answer Given:{initial_answer}\n\n Review your previous answer and find problems with your answer"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [47]:
rc_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [48]:
chain2 = rc_prompt | model | StrOutputParser()

In [49]:
constructive_criticism = chain2.invoke({"question": initial_question, "initial_answer":fake_initial_ai_answer})
constructive_criticism

'The calculation for the additional tennis balls is incorrect. Two cans of tennis balls with 3 tennis balls each would result in 2 x 3 x 3 = 18 additional tennis balls, not 6. Therefore, the total number of tennis balls Roger has now is 5 + 18 = 23, not 9.'

## Part 3 - The Improvement

In [50]:
template="You are a helpful assistant that reviews answers and critiques based on the original question given and write a new improved final answer."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="### Question:\n\n{question}\n\n ###Answer Given:{initial_answer}\n\n \
###Constructive Criticism:{constructive_criticism}\n\n Based on the problems you found, improve your answer.\n\n### Final Answer:"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [51]:
improvement_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [52]:
chain3 = improvement_prompt | model | StrOutputParser()

In [53]:
final_result = chain3.invoke({"question": initial_question,
                              "initial_answer":fake_initial_ai_answer,
                              "constructive_criticism": constructive_criticism})

final_result

'Roger initially has 5 tennis balls. He bought 2 cans of tennis balls, and each can has 3 tennis balls, so he has 2 x 3 = 6 additional tennis balls. Therefore, the total number of tennis balls Roger has now is 5 + 6 + 9.'

## Combined Chain

In [54]:
from operator import itemgetter

In [55]:

chain1 = chat_prompt | model | StrOutputParser()

critque_chain = {"question": itemgetter("question"),
                 "initial_answer": chain1 } | rc_prompt | model | StrOutputParser()

chain3 = {"question": itemgetter("question"),
          "initial_answer": chain1,
          "constructive_criticism": critque_chain} | improvement_prompt | model | StrOutputParser()

In [56]:
chain3.invoke({"question":"Write an sms message to say I am tired"})

'A better SMS message to say you are tired could be "I\'m tired". This response is short, simple, and to the point, which is appropriate for an SMS message format.'

In [57]:
langchain.debug = True

In [58]:
chain3.invoke({"question":"Write an sms message to say I am tired"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Write an sms message to say I am tired"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap] Entering Chain run with input:
{
  "input": {
    "question": "Write an sms message to say I am tired"
  }
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableLambda] Entering Chain run with input:
{
  "question": "Write an sms message to say I am tired"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": "Write an sms message to say I am tired"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 4:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Write an sms message to say I am tired"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 4:chain:RunnableSequence > 5:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "quest